In [ ]:
# pip install CHAID

In [ ]:
from CHAID import Tree
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Leitura dos dados
telefonia = pd.read_table("C:/_Ayu/0-Fia/_2020/TURMAS/EA_EAD_ago20/\Python/Aula2/Telefonia_AD.txt")
telefonia.head()

In [ ]:
telefonia.dtypes

In [ ]:
telefonia.shape

In [ ]:
telefonia.describe().transpose()

In [ ]:
# Verificando quantidade de missings
telefonia.isnull().sum()

In [ ]:
# Tratamento da variável Minutos_realizados_T0
telefonia.Minutos_realizados_T0 = telefonia.Minutos_realizados_T0.fillna(0)

In [ ]:
# Como a variável explicativa também deve ser categórica, 
# vamos segmentar a Idade em quartil
telefonia['Minutos_realizados_T0_q'] = pd.qcut(telefonia.Minutos_realizados_T0, 4)
telefonia['Tempo_casa_q'] = pd.qcut(telefonia.Tempo_casa, 4)
telefonia['Qtd_retencao_6meses_q'] = pd.qcut(telefonia.Qtd_retencao_6meses, 4, duplicates='drop')
telefonia['Qtd_prod_q'] = pd.qcut(telefonia.Qtd_prod, 4, duplicates='drop')

In [ ]:
# Tabela Bidimensional: covariável x resposta
minutos_tab = pd.crosstab(telefonia["Minutos_realizados_T0_q"],telefonia["resposta"],margins=True)
minutos_tab

In [ ]:
tempo_casa_tab = pd.crosstab(telefonia["Tempo_casa_q"],telefonia["resposta"],margins=True)
tempo_casa_tab

In [ ]:
qtd_retencao_tab = pd.crosstab(telefonia["Qtd_retencao_6meses_q"],telefonia["resposta"],margins=True)
qtd_retencao_tab

In [ ]:
qtd_prod_tab = pd.crosstab(telefonia["Qtd_prod_q"],telefonia["resposta"],margins=True)
qtd_prod_tab

## Modelo - Árvore de Decisão

In [ ]:
# Transformando a variável resposta em categórica
telefonia['resposta_cat'] = telefonia.resposta.astype('category')

In [ ]:
var_explicativas = telefonia[[
    'Minutos_realizados_T0_q',
    'Tempo_casa_q', 
    'Qtd_retencao_6meses_q',
    'Qtd_prod_q']]

var_resposta = telefonia['resposta_cat']

In [ ]:
# Constrói o modelo de árvore
modelo = Tree.from_numpy(
    var_explicativas.to_numpy(), 
    var_resposta.to_numpy(), 
    split_titles=['Minutos_realizados_T0_q',
                  'Tempo_casa_q', 
                  'Qtd_retencao_6meses_q',
                  'Qtd_prod_q'], 
    min_child_node_size=2)

In [ ]:
modelo.print_tree()

In [ ]:
modelo.classification_rules()

In [ ]:
# Salvando na base os nós
telefonia['node'] = modelo.node_predictions()

In [ ]:
# Salvando a taxa de resposta por nó
probs = telefonia.groupby(['node']).agg({'resposta':'mean'}).reset_index()

In [ ]:
# Marcando se o nó é propenso
probs['propenso'] = np.where(probs['resposta'] >= telefonia['resposta'].mean(), 1, 0)

In [ ]:
probs = probs.rename(columns={"resposta":"prob"})
probs

In [ ]:
telefonia = telefonia.merge(probs, how='left', on='node')

In [ ]:
tabela_desempenho = pd.crosstab(telefonia['resposta'], telefonia['propenso'])

In [ ]:
tabela_desempenho

In [ ]:
acuracia = (tabela_desempenho[0][0] + tabela_desempenho[1][1])/tabela_desempenho.sum().sum()
acuracia

In [ ]:
sensibilidade = (tabela_desempenho[1][1])/(tabela_desempenho[1][1] + tabela_desempenho[0][1])
sensibilidade

In [ ]:
especificidade = (tabela_desempenho[0][0])/(tabela_desempenho[0][0] + tabela_desempenho[1][0])
especificidade